In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
sns.set(font='SimHei')

In [ ]:
train = pd.read_csv('../input/bike-sharing-demand/train.csv')
test = pd.read_csv('../input/bike-sharing-demand/test.csv')
train.info()
test.info() ## 没有空值

In [ ]:
train.head()

In [ ]:
#查看是否符合高斯分布
train['count'].describe()

均值191，标准差181，50%分位数是145，75%分位数是284，最大值977，说明右侧存在长尾。去除掉异常值，并取log处理，观察结果。

In [ ]:
#把超出3倍标准差的数据，共147个剔除
train = train.loc[np.abs(train['count']-train['count'].mean()) < (3*train['count'].std())]
#对剔除异常值后的count和count_log进行比较
train['count_log'] = np.log(train['count'])
f, [ax1, ax2] = plt.subplots(1,2, figsize=(15,6))
sns.distplot(train['count'], ax=ax1)
ax1.set_title('Distribution of count')
sns.distplot(train['count_log'], ax=ax2)
ax2.set_title('Distribution of count_log')

左边是去除异常值后count分布，右边是取log后的

In [ ]:
#合并数据
df = train.append(test, ignore_index=True)
#整理列顺序
df = pd.DataFrame(df, columns=train.columns)
df.tail()

In [ ]:
#将datetime分解为多种时间格式
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
#细化
# 去掉weekday，方便看每周的变化情况
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.weekday

In [ ]:
#查看天气状况
f, ax = plt.subplots(2,2, figsize=(12,10))
sns.distplot(df.temp, ax=ax[0,0])
ax[0,0].set_title('Distribution of temp')
sns.distplot(df.atemp, ax=ax[0,1])
ax[0,1].set_title('Distribution of atemp')
sns.distplot(df.humidity, ax=ax[1,0])
ax[1,0].set_title('Distribution of humidity')
sns.distplot(df.windspeed, ax=ax[1,1])
ax[1,1].set_title('Distribution of windspeed')

In [ ]:
#风速为0的数据偏多，且有空缺，采用随机森林的方法填充异常值
wind_0 = df[df['windspeed']==0]
wind_not0 = df[df['windspeed']!=0]
y_label = wind_not0['windspeed']

In [ ]:
#猜测风速和天气以及时间都有关
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
windcolunms = ['season', 'weather', 'temp', 'atemp', 'humidity', 'hour', 'month']
model.fit(wind_not0[windcolunms], y_label.astype('int'))
pred_y = model.predict(wind_0[windcolunms])
#预测结果填充
wind_0['windspeed'] = pred_y
df_rfw = wind_not0.append(wind_0)
df_rfw.reset_index(inplace=True)
df_rfw.head()

In [ ]:
df_rfw = df_rfw.drop('index', axis=1)
#查看处理后的风速情况
f, ax = plt.subplots(figsize=(8,5))
sns.distplot(df_rfw['windspeed'], ax=ax)
ax.set_title('Distribution of handled windspeed')

In [ ]:
#查看各组数据和count的相关性
f, ax = plt.subplots(figsize=(20,16))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(df_rfw[df_rfw['count'].notnull()].corr(), cmap=cmap, ax=ax, annot=True, lw=.1)


In [ ]:
#count和temp/atemp/hour有较明显正相关，和humidity有较明显负相关
df_rfw[df_rfw['count'].notnull()].corr()['count'].sort_values(ascending=False)

查看count与各元素间的变动关系

In [ ]:
#时间数据
count_columns = ['count', 'registered', 'casual']
f, [ax1, ax2] = plt.subplots(1,2, figsize=(15,5))
df_rfw.groupby(['year','season'])[count_columns].mean().plot.line(ax=ax1)
df_rfw.groupby('season')[count_columns].mean().plot.line(ax=ax2)
ax2.set_xticks(range(1,5))

registered和casual的用户均呈现上升的趋势，夏季用户租赁量增速最快，冬天租赁量回退，春秋租赁量缓慢增加。

In [ ]:
f, ax = plt.subplots(figsize=(10,5))
df_rfw.groupby('hour')[count_columns].mean().plot.line(ax=ax)
ax.set_title('租车量在一天内的变化', fontsize=15)

registered用户存在明显的早晚高峰，对应上班时间，中午午饭时间也有小峰值，casual用户则是在10-20点有灵活租赁。

In [ ]:
f, [ax1,ax2] = plt.subplots(1,2,figsize=(15,5))
df_rfw_w = df_rfw.loc[df_rfw['workingday']==1]
df_rfw_notw = df_rfw.loc[df_rfw['workingday']==0]
df_rfw_w.groupby('hour')[count_columns].mean().plot.line(ax=ax1)
df_rfw_notw.groupby('hour')[count_columns].mean().plot.line(ax=ax2)
ax1.set_title('工作日租车量在一天内的变化', fontsize=15)
ax2.set_title('非工作日租车量在一天内的变化', fontsize=15)

将工作日与非工作日分开看，可以看到在工作日有更明显的早晚高峰，在非工作日两种用户的租赁趋势相同

In [ ]:
f, ax = plt.subplots(figsize=(10,5))
df_rfw.groupby('weekday')[count_columns].mean().plot.line(ax=ax)
ax.set_title('租赁数量的周变化曲线')

通过周变化曲线，发现registered用户在周一到周五的租赁数量稳定，周末两天减少20%，casual用户在周末租赁数量是工作日的两倍

In [ ]:
#天气数据
f, ax = plt.subplots(figsize=(10,5))
df_rfw.groupby('weather')[count_columns].mean().plot.line(ax=ax)
ax.set_title('租赁数量随天气的变化曲线')
ax.set_xticks(range(1,5))

天气条件越不好，租赁人数越少，和预期相同，但是天气为4，也就是暴雨暴雪时反而租赁量很高，需要具体查看分析。

In [ ]:
df_rfw.loc[df_rfw['weather']==4]

天气情况为4时，只有一组数据，而且是周一的18点，是一天的租赁最高峰，相比正常情况下400+的租赁，只有164，说明天气对租赁量影响是很大的

In [ ]:
f, [ax1, ax2] = plt.subplots(2,1,figsize=(12,10))
df_rfw.groupby('temp')[count_columns].mean().plot.line(ax=ax1)
ax1.set_title('租赁数量随气温的变化曲线')
df_rfw.groupby('atemp')[count_columns].mean().plot.line(ax=ax2)
ax2.set_title('租赁数量随体感气温的变化曲线')

租赁数量随温度升高，呈上升趋势，气温36℃左右，体感温度40℃左右达到最大值。且体感温度和气温的曲线走势接近，只是延后4℃。

气温38℃以上有异常，需根据具体数据确定

In [ ]:
df_rfw.loc[df_rfw['temp']>=38].head(10)
##在2012-07-07一天11:00-18:00存在长时间高租赁，可能有集体活动，具有偶然性

In [ ]:
#因为temp和atemp相关系数为0.99，将temp和atemp合并
df_rfw['new_temp'] = (df_rfw['temp'] + df_rfw['atemp'])/2
#湿度的影响
f, ax = plt.subplots(figsize=(10,5))
df_rfw.groupby('humidity')[count_columns].mean().plot.line(ax=ax)
ax.set_title('租赁数量随湿度的变化曲线')
#随湿度的增加，租赁量缓慢下降。

In [ ]:
#风速跨度比较大，所以对其进行分组
df_rfw['wind_class'] = pd.cut(df_rfw['windspeed'], 9)
f, [ax1, ax2] = plt.subplots(2,1,figsize=(12,10))
df_rfw.groupby('windspeed')[count_columns].mean().plot.line(ax=ax1)
ax1.set_title('租赁数量随风速的变化曲线')
df_rfw.groupby('wind_class')[count_columns].mean().plot.line(ax=ax2)
ax2.set_title('租赁数量随风速等级的变化曲线')
#风速较低时，对租赁量的影响不大，风速超过45时，租赁量迅速缩小，和异常天气时的情况类似。

In [ ]:
#查看风速>51的数据，均值偏高可以归于异常值。
df_rfw.loc[df_rfw['windspeed']>=51]

In [ ]:
#查看处理后的数据
df_rfw.info()

In [ ]:
#对分类数据进行one-hot编码
season_dummy=pd.get_dummies(df_rfw['season'],prefix='season')
weather_dummy=pd.get_dummies(df_rfw['weather'],prefix='weather')
month_dummy=pd.get_dummies(df_rfw['month'],prefix='month')
df_rfw1 = pd.concat([df_rfw,season_dummy,weather_dummy,month_dummy],axis=1)
df_rfw1.head()

In [ ]:
#分离训练集和测试集
df_train=df_rfw1[df_rfw1['count'].notnull()].sort_values('datetime',ascending=True)
df_test=df_rfw1[df_rfw['count'].isnull()].sort_values('datetime',ascending=True)
#丢弃掉不要的列
drop_columns=['datetime','season','weather','casual','registered','count','month','temp','atemp','wind_class']
df_train=df_train.drop(columns=drop_columns,axis=1)
df_test=df_test.drop(columns=drop_columns,axis=1)
count_log=df_train['count_log']
df_train.drop('count_log',axis=1,inplace=True)
df_test.drop('count_log',axis=1,inplace=True)
df_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=2000,random_state=42)
model.fit(df_train,count_log)
pred=model.predict(df_test)

In [ ]:
#评估准确度
model.score(df_train, count_log)

In [ ]:
#转化为可提交的数据格式
pred_exp=np.exp(pred)
pred_exp=pd.Series(pred_exp,name='count')
pred_concat=pd.concat([test['datetime'],pred_exp],axis=1)
pred_concat.to_csv('bike_submission.csv_3_online', index=False)